In [3]:
import cadquery as cq
from cadquery import Workplane, Assembly, Location, Vector, Color, exporters
# from cq_server.ui import ui, show_object
# from ocp_vscode import *

from ocp_vscode import show, show_object, reset_show, set_port, set_defaults, get_defaults
set_port(3939)

# cadquery
# cq-editor
# cadquery-server command
# cq-server run -p 8080 --ui-theme dark --ui-glass --ui-hide all
# cq-server run -p 8080 --ui-theme dark --ui-hide all
# http://192.168.1.174:8080

# colors: https://techoverflow.net/2019/06/14/overview-of-all-standard-colors-available-in-opencascade/


In [2]:
assy_ci_bot = cq.Assembly() # CI bottom layer (contact pad + measurement pad)
assy_di_mid = cq.Assembly() # DI middle layer
assy_ci_top = cq.Assembly() # CI top layer (contact pad + measurement pad)
assy_ci_extra = cq.Assembly() # text on side with sizes + cross anchors
assy_di_extra = cq.Assembly() # DI thickness coupon

# NOTE: CadQuery units in mm (1500.0 = 1500mm)
mm_to_micron = 0.001

# Parametric parameters
# Widths - (top to bottom) of connection b/t nodes
a_dim = list(range(50, 160, 10))
a_dim.extend(range(200, 410, 50))
a_dim.append(500)
a_dim.append(1000)
thickness_dim = [20, 30, 40, 50, 70, 100, 150, 200, 300, 400]
combined_dim = [(a, t) for a in a_dim for t in thickness_dim]
# print(combined_dim)

# Text Dimensions
text_size = 750.0 * mm_to_micron

# Cross Dimensions
cross_rect_height = 400 * mm_to_micron
cross_rect_width = 100 * mm_to_micron
cross_corner_offset = 1000 * mm_to_micron

# Print thickness (CI) (static, 20µm)
ci_thickness = 20.0 * mm_to_micron

# Trace pad dimensions (static, 300x200µm)
# pad_height = 300.0 * mm_to_micron
# pad_width = 200.0 * mm_to_micron
pad_height = 1000.0 * mm_to_micron
pad_width = 1000.0 * mm_to_micron

for row, a in enumerate(a_dim):
    for col, t in enumerate(thickness_dim):

        """
        VARIABLES
        """

        # Offsets for each new parametric object (5mm x 2.2mm)
        # y_shift_increment = 600
        # x_shift_increment = 1800
        # y_shift_increment = 2200
        # x_shift_increment = 3200
        y_shift_increment = 2200
        x_shift_increment = 5000
        y_shift = (y_shift_increment * row) * mm_to_micron
        x_shift = (x_shift_increment * col) * mm_to_micron

        # CI box dimensions (parameterized)
        ci_height = a * mm_to_micron
        ci_width = a * mm_to_micron

        # CI trace connector dimensions (300x100µm)
        # NOTE: 200 (max(a_dim)) here is the largest a / 2; inversely proportional, as a increases, length should decrease
        connector_height = (300.0 + (max(a_dim) / 2 - (a / 2))) * mm_to_micron
        connector_width = 100.0 * mm_to_micron

        # DI (middle layer) dimensions
        # 100 micron = 50 micron over on each side
        di_mid_height = ci_height + (200.0 * mm_to_micron)
        di_mid_width = ci_width + (200.0 * mm_to_micron)
        di_thickness = t * mm_to_micron

        """
        SHAPES
        """

        # Layer 1 - CI at bottom
        base_CI = Workplane()
        base_CI = base_CI.box(
            ci_height,
            ci_width,
            ci_thickness
        )

        # Layer 1 - CI connector to contact pad
        base_CI_connector = Workplane()
        base_CI_connector = base_CI_connector.box(
            connector_height,
            connector_width,
            ci_thickness
        )

        # Layer 1 - CI connector to contact pad (to be cut from DI overfill)
        base_CI_connector_cut = Workplane(
            origin=(
                -(ci_height / 2 + connector_height / 2),
                0,
                0
            ))
        base_CI_connector_cut = base_CI_connector_cut.box(
            connector_height,
            connector_width,
            ci_thickness
        )

        # Layer 1 - CI contact pad
        base_CI_contact_pad = Workplane()
        base_CI_contact_pad = base_CI_contact_pad.box(
            pad_height,
            pad_width,
            ci_thickness
        )

        # Layer 2 - DI middle layer
        di_middle_layer = Workplane()
        di_middle_layer = di_middle_layer.box(
            di_mid_height,
            di_mid_width,
            di_thickness
        )

        # Layer 2 - DI middle layer overfill
        di_middle_layer_overfill = Workplane()
        di_middle_layer_overfill = di_middle_layer_overfill.box(
            di_mid_height,
            di_mid_width,
            ci_thickness
        )
        di_middle_layer_overfill = di_middle_layer_overfill.cut(base_CI)
        di_middle_layer_overfill = di_middle_layer_overfill.cut(
            base_CI_connector_cut)

        # Layer 2 - CI contact suspender
        mid_layer_suspender = Workplane()
        mid_layer_suspender = mid_layer_suspender.box(
            pad_height + connector_height,
            pad_width + connector_width / 2,
            ci_thickness + di_thickness
        )

        # Layer 3 - Upper CI
        upper_base_CI = Workplane()
        upper_base_CI = upper_base_CI.box(
            ci_height,
            ci_width,
            ci_thickness
        )

        # Layer 3 - CI connector to contact pad
        upper_base_CI_connector = Workplane()
        upper_base_CI_connector = upper_base_CI_connector.box(
            connector_height,
            connector_width,
            ci_thickness
        )

        # Layer 3 - CI contact pad
        upper_base_CI_contact_pad = Workplane()
        upper_base_CI_contact_pad = upper_base_CI_contact_pad.box(
            pad_height,
            pad_width,
            ci_thickness
        )

        # Text - indicates a (box dimensions) and DI thickness
        di_thickness_text = Workplane()
        di_thickness_text = di_thickness_text.text(
            str(t), text_size, ci_thickness, fontPath="/Users/avocado/Documents/Documents - A’s MacBook Pro/School/AIMS Lab/_gdspy/4_2nd_design/square-deal.ttf")
            # str(t), text_size, ci_thickness, font="Impact")

        ci_dim_text = Workplane()
        ci_dim_text = ci_dim_text.text(
            str(a), text_size, ci_thickness, fontPath="/Users/avocado/Documents/Documents - A’s MacBook Pro/School/AIMS Lab/_gdspy/4_2nd_design/square-deal.ttf")
            # str(a), text_size, ci_thickness, font="Impact")

        # Corner Crosses - for prober alignment
        cross_rect1 = Workplane()
        cross_rect1 = cross_rect1.box(
            cross_rect_height, cross_rect_width, ci_thickness)
        cross_rect2 = Workplane()
        cross_rect2 = cross_rect2.box(
            cross_rect_width, cross_rect_height, ci_thickness)
        cross_shape = cross_rect1.union(cross_rect2)

        """
        LAYER ASSEMBLY
        """

        # Layer 1 Assembly
        assy_ci_bot = assy_ci_bot.add(
            base_CI,
            loc=Location(Vector(
                x_shift,
                y_shift,
                0
            )),
            # color=Color("red"))
            color=Color("gray"))
        assy_ci_bot = assy_ci_bot.add(
            base_CI_connector,
            loc=Location(Vector(
                x_shift + -(ci_height / 2 + connector_height / 2),
                y_shift,
                0
            )),
            # color=Color("green"))
            color=Color("gray"))
        assy_ci_bot = assy_ci_bot.add(
            base_CI_contact_pad,
            loc=Location(Vector(
                x_shift + -(ci_height / 2 + connector_height + pad_height / 2),
                y_shift,
                0
            )),
            # color=Color("blue")
            color=Color("gray")
        )

        # Add DI height coupon (just DI thickness) at the end
        if (row == len(a_dim) - 1):
            assy_di_extra = assy_di_extra.add(
                di_middle_layer,
                loc=Location(Vector(
                    x_shift,
                    # 600 = y_shift increment
                    y_shift + (y_shift_increment * mm_to_micron),
                    (di_thickness) / 2
                )),
                color=Color("Quantity_NOC_DARKGOLDENROD2")
                # color=Color("purple")
            )

            # Add text for DI thickness
            assy_ci_extra = assy_ci_extra.add(
                di_thickness_text,
                loc=Location(Vector(
                    x_shift,
                    # 1200 = y_shift increment * 2 (DI coupon first)
                    y_shift + ((y_shift_increment * 2) * mm_to_micron),
                    0
                )),
                color=Color("gray")
                # color=Color("purple")
            )

        # Add text for a_dim
        if (col == len(thickness_dim) - 1):
            assy_ci_extra = assy_ci_extra.add(
                ci_dim_text,
                loc=Location(Vector(
                    # 1800 = x_shift increment
                    x_shift + (x_shift_increment * mm_to_micron),
                    y_shift,
                    0
                )),
                color=Color("gray")
                # color=Color("purple")
            )

        # Add crosses to each corner
        if (col == 0 and row == 0):  # bot-left
            assy_ci_extra = assy_ci_extra.add(
                cross_shape,
                loc=Location(Vector(
                    # dim from layer 1 contact pad
                    x_shift + -(ci_height / 2 + connector_height + \
                                pad_height / 2) - cross_corner_offset,
                    y_shift - cross_corner_offset,
                    0
                )),
                color=Color("gray")
            )
        elif (col == 0 and row == len(a_dim) - 1):  # top-left
            assy_ci_extra = assy_ci_extra.add(
                cross_shape,
                loc=Location(Vector(
                    # dim from layer 1 contact pad
                    x_shift + -(ci_height / 2 + connector_height + \
                                pad_height / 2) - cross_corner_offset,
                    y_shift + cross_corner_offset,
                    0
                )),
                color=Color("gray")
            )
        elif (col == len(thickness_dim) - 1 and row == 0):  # bot-right
            assy_ci_extra = assy_ci_extra.add(
                cross_shape,
                loc=Location(Vector(
                    (x_shift + ci_height / 2 + connector_height + pad_height /
                     2) + cross_corner_offset,  # dim from layer 3 contact pad
                    y_shift - cross_corner_offset,
                    0
                )),
                color=Color("gray")
            )
        elif (col == len(thickness_dim) - 1 and row == len(a_dim) - 1):  # top-right
            assy_ci_extra = assy_ci_extra.add(
                cross_shape,
                loc=Location(Vector(
                    (x_shift + ci_height / 2 + connector_height + pad_height /
                     2) + cross_corner_offset,  # dim from layer 3 contact pad
                    y_shift + cross_corner_offset,
                    0
                )),
                color=Color("gray")
            )

        # Layer 2 Assembly
        assy_di_mid = assy_di_mid.add(
            di_middle_layer,
            loc=Location(Vector(
                x_shift,
                y_shift,
                (ci_thickness + di_thickness) / 2  # Z

            )),
            color=Color("Quantity_NOC_DARKGOLDENROD2")
        )

        assy_di_mid = assy_di_mid.add(
            di_middle_layer_overfill,
            loc=Location(Vector(
                x_shift,
                y_shift,
                0  # Z

            )),
            color=Color("Quantity_NOC_DARKGOLDENROD2")
        )

        assy_di_mid = assy_di_mid.add(
            mid_layer_suspender,
            loc=Location(Vector(
                # (1540 + 300 + 500)/ 2 = 1170
                x_shift + (di_mid_height + connector_height + pad_height) / 2,
                # -ci_width / 4,
                y_shift,
                (di_thickness / 2)
            )),
            color=Color("Quantity_NOC_DARKGOLDENROD2")
        )

        # Layer 3 Assembly
        assy_ci_top = assy_ci_top.add(
            upper_base_CI,
            loc=Location(Vector(
                x_shift,
                y_shift,
                di_thickness + ci_thickness
            )),
            color=Color("gray")
        )
        assy_ci_top = assy_ci_top.add(
            upper_base_CI_connector,
            loc=Location(Vector(
                x_shift + ci_height / 2 + connector_height / 2,
                y_shift,
                di_thickness + ci_thickness

            )),
            color=Color("gray"))
        assy_ci_top = assy_ci_top.add(
            upper_base_CI_contact_pad,
            loc=Location(Vector(
                x_shift + ci_height / 2 + connector_height + pad_height / 2,
                y_shift,
                di_thickness + ci_thickness

            )),
            color=Color("gray")
        )

        # if col == 2:
        #     break

    # break

# Show each of the 3 layers
show_object(assy_ci_bot)
show_object(assy_di_mid)
show_object(assy_ci_top)
show_object(assy_ci_extra)
show_object(assy_di_extra)

# show(assy_ci_bot)
# show(assy_di_mid)
# show(assy_ci_top)
# show(assy_ci_extra)
# show(assy_di_extra)



++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++cccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc

In [3]:
# exporters.export(assy, 'design_two.stl')
assy_ci_bot.save('assy_ci_bot.step')
assy_di_mid.save('assy_di_mid.step')
assy_ci_top.save('assy_ci_top.step')
assy_ci_extra.save('assy_ci_extra.step')
assy_di_extra.save('assy_di_extra.step')
